# TensorFlow code for a Recurrent Neural Network with Sequence Input

In this section we will go through the code for a slightly different Recurrent Neural Network in TensorFlow.

### The aim of the network

For a given a binary string we need to determine the count of 1s in the string. For example, “01010” has 2 ones.

The input for our network will be a string of length twenty that contains 0s and 1s and the output will be a single number, between 0 and 20, which represents the number of ones in the string.

First of all we set up the required imports.

In [ ]:
from __future__ import division, print_function, absolute_import
import os
from time import time
from datetime import datetime
import numpy as np
import random
from random import shuffle
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)


Here are the relevant training and network parameters and graph input for context.

In [ ]:
# Training Parameters

learning_rate = 0.0001 # Initial learning rate
training_epochs = 2500 # Number of epochs to train
batch_size = 100 # Number of sequences per batch
display_step = 50 # How often to output model metrics during training

num_input = 12 # data input length
num_hidden = 24 # hidden layer num of features
num_classes = num_input + 1 # Number of output classes
training_examples = 500 # Number of examples to use for training

X = tf.placeholder("float", [None, num_input,1], name='X')
Y = tf.placeholder("float", [None, num_classes], name='Y')

Generate synthetic data

In [ ]:
train_input = ['{0:0{length}b}'.format(i, length=num_input) for i in range(2**num_input)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

## Gen train target
train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count += 1
    temp_list = ([0]*num_classes)
    temp_list[count] = 1
    train_output.append(temp_list)

## Train/Test split
test_input = train_input[training_examples:]
test_output = train_output[training_examples:]
train_input = train_input[:training_examples]
train_output = train_output[:training_examples]

Initialise weights and biases for the network.

In [ ]:
weights = tf.Variable(tf.truncated_normal([num_hidden, int(Y.get_shape()[1])]), name='weights')
biases = tf.Variable(tf.constant(0.1, shape=[Y.get_shape()[1]]), name='biases')

### Model Creation

`tf.nn.rnn_cell.LSTMCell` - Default LSTM recurrent network cell.

`tf.nn.dynamic_rnn` = Creates a recurrent neural network with the specified RNNCell cell.

`tf.transpose` - Transposes a tensor. Permutes the dimensions according to perm.

`tf.gather` - Gather slices from params axis axis according to indices

In [ ]:
def RNN(x, weights, biases):

    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True, name='lstm_cell')

    # Get lstm cell output
    val, _ = tf.nn.dynamic_rnn(lstm_cell, X, dtype=tf.float32)

    # Transpose tensor dimensions
    val = tf.transpose(val, perm=[1, 0, 2], name='transpose')

    # Gather sparse to dense
    last = tf.gather(params=val, indices=tf.Variable(int(val.get_shape()[0]) - 1, name='gather_indices'), name='gather')

    # Linear activation, using rnn inner loop last output
    return tf.matmul(last, weights, name='out') + biases

### Define loss and optimizer

In the following snippet we define our loss operation, optimiser and initialise our global variables.

`tf.reduce_sum` - Computes the sum of elements across dimensions of a tensor.

`tf.train.AdamOptimizer` - Optimizer that implements the Adam algorithm algorithm.

`optimizer.minimize` - Takes care of both computing the gradients and applying them with respect to `loss_op`.

In [ ]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits, name='prediction')

loss_op = -tf.reduce_sum(Y * tf.log(tf.clip_by_value(prediction,1e-10,1.0)), name='loss_op')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op, name='train_op')

### Define evaluation metrics

In [ ]:
incorrect_pred = tf.not_equal(tf.argmax(Y, 1), tf.argmax(prediction, 1), name='incorrect_pred')
error = tf.reduce_mean(tf.cast(incorrect_pred, tf.float32), name='error')

Initialize the variables (i.e. assign their default value)

In [ ]:
init = tf.global_variables_initializer()

### Setup tensorboard

In [ ]:
# Define writer for tensorbord log output
writer = tf.summary.FileWriter(os.path.join(os.getcwd(),"rnn-seq-tb-" + str(datetime.fromtimestamp(time())) ), graph=tf.get_default_graph())

# Define and name tensorboard histograms
tf.summary.histogram("loss", loss_op)
tf.summary.histogram("error", error)

# Create a summary to monitor cost tensor
#tf.summary.scalar("loss", loss_op)
# Create a summary to monitor accuracy tensor
#tf.summary.scalar("error", error)
# Merge all summaries into a single op

# Merge all summaries into a single output
merged_summary_op = tf.summary.merge_all()

### Train and evaluate the model

In [ ]:
from matplotlib import pyplot as plt
no_of_batches = int(len(train_input)) // batch_size
with tf.Session() as sess:
    sess.run(init)
    for step in range(training_epochs):
        ptr = 0
        for j in range(no_of_batches):
            inp, out = train_input[ptr:ptr + batch_size], train_output[ptr:ptr + batch_size]
            ptr += batch_size
            sess.run(train_op, {X: inp, Y: out})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and error
            loss, err, summary = sess.run([loss_op, error, merged_summary_op], {X: inp, Y: out})
            writer.add_summary(summary, step)
            
            print("Step " + str(step) + ", Minibatch Loss = " + \
                    "{:.4f}".format(loss) + ", Training Error = " + \
                    "{:.5f}".format(err))

    print("Testing Error:", "{:.5f}".format(sess.run(error,{X: test_input, Y: test_output})))
    pred = tf.nn.softmax(logits)
    # Build confusion matrix from ground truth labels and model predictions
    conf_mat = tf.confusion_matrix(tf.argmax(Y, 1), tf.argmax(pred, 1)).eval({X: test_input, Y: test_output})
    %matplotlib inline
    # Plot matrix
    plt.matshow(conf_mat)
    plt.ylabel('Real Class')
    plt.xlabel('Predicted Class')
    plt.show()
    sess.close()

### Setup tensorboard using an ngrok tunnel

In [ ]:
import time
import subprocess
import os
import signal

def get_process_pid(pstring):
    pid = None
    for line in os.popen("ps ax | grep " + pstring + " | grep -v grep | grep -v defunct"):
        fields = line.split()
        pid = fields[0]
    return pid

LOG_DIR = os.getcwd()
NG_DIR = LOG_DIR
# Uncomment if running locally
#NG_DIR = os.path.dirname(LOG_DIR)
NG_ZIP = os.path.join(NG_DIR, 'ngrok-stable-linux-amd64.zip')
NG_BIN = os.path.join(NG_DIR, 'ngrok')

# Download ngrok binary
if not os.path.isfile(NG_ZIP):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip \
        -P {NG_DIR}
if not os.path.isfile(NG_BIN):        
    !unzip -o {NG_DIR}/ngrok-stable-linux-amd64.zip -d {NG_DIR}

# If tensorboard is alredy running kill it and restart with the correct logdir
tb_pid = get_process_pid('tensorboard')
if tb_pid:
    print("Killing old tensorboard")
    os.kill(int(tb_pid), signal.SIGKILL)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
tb_pid = get_process_pid('tensorboard')
print ("Started tensorboard with pid %s" % tb_pid)

# If ngrok is alredy running do nothing
ng_pid = get_process_pid('ngrok')
if not ng_pid:
    proc = subprocess.Popen(['%s/ngrok' % NG_DIR , 'http', '6006'])
    print ("Started ngrok with pid %s" % proc.pid)
    time.sleep(5)
else:
    print ("ngrok alredy runing")
ng_pid = get_process_pid('ngrok')

# Get ngrok link
try:
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
        "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
except:
    print("Error getting ngrok link. Retrying...")
    time.sleep(5)
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
        "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# Cleanup
#procs = [tb_pid, ng_pid]
#[os.kill(int(x), signal.SIGKILL) for x in procs if x is not None]
#!rm -rf rnn-seq-tb-*

### Experiment
Now try experimenting with the model. What effects do you see when changing the model parameters?
 - learning_rate
 - training_epochs
 - batch_size
 - num_hidden
 - num_input
 - training_examples
 
Try adding additional LSTM cells to the model.
 - Hint: Use a stacked cell - ```lstm_cell = rnn.MultiRNNCell( <list_of_lstm_cells> )```

## End of RNN-SEQ Notebook